In [1]:
from IPython.display import display, HTML, Math, Markdown

### APT_keyの登録

`.env`ファイルを作成して以下のように登録。あるいは、本ファイル内に直接記入しても良い。

```
OPENROUTER_API_KEY=your_api_key_here
```

In [2]:

import os
from dotenv import load_dotenv

from typing import List, Dict

load_dotenv()

# api_key = "your_api_key_here"
# api_key = os.environ.get("OPENAI_API_KEY")
# base_url = "https://api.openai.com/v1"

# api_key = "your_api_key_here"
api_key = os.environ.get("OPENROUTER_API_KEY")
base_url = "https://openrouter.ai/api/v1"

### 動作確認

openaiの仕様に準じているみたいです。`import openai`が使えます。

In [3]:
import openai

client = openai.OpenAI(api_key=api_key, base_url=base_url)

In [4]:
msgs:List[Dict[str, str]] = [
    {"role": "system", "content": "あなたは親切なアシスタントです。"},
    {"role": "user", "content": "こんにちは！"},
]

# GPTのテスト
response = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=msgs,
)
print(response.choices[0].message.content)

こんにちは！何かお手伝いできることはありますか？気軽に話しかけてくださいね！


# persona seed の作り方 

`proj-persona/PersonaHub` (license: cc-by-nc-sa-4.0)  
https://huggingface.co/datasets/proj-persona/PersonaHub  

persona
```text
A scientist who specializes in virology and the study of ancient viruses. This persona is interested in the potential dangers of thawing permafrost and the revival of ancient viruses, and is likely to be part of a research team studying the potential impact of these viruses on human and animal health. This persona may also be familiar with the work of Jean-Michel Claverie, the lead author of the study.

```

> ウイルス学と古代ウイルスの研究を専門とする科学者。このペルソナは、永久凍土の融解と古代ウイルスの復活の潜在的な危険性に関心を持っており、これらのウイルスが人間と動物の健康に及ぼす潜在的な影響を研究する研究チームの一員である可能性が高い。また、この研究の筆頭著者であるジャン＝ミシェル・クラヴェリー氏の研究にも精通している可能性がある。

例えば、上記personaの場合、`{"subject":biology, "group":microbiology, "category": ancient virology}`のような属性があるという前提で、幅広い範囲でその組み合わをLLMに考えてもらう。
```
{
  "biology": {
    "microbiology": ["virology", "bacteriology", "mycology", "parasitology", "environmental microbiology", "industrial microbiology", "microbial genetics"],
    "genetics": ["molecular genetics", "population genetics", "human genetics", "epigenetics", "quantitative genetics", "evolutionary genetics", "genomic medicine"],
  ...,
    "art and architecture": ["art history", "architectural theory", "visual culture", "digital art", "conservation science", "museum studies", "critical heritage studies"]
  }
}
```
  
↓  
続いて、`{"subject":biology, "group":microbiology}`のような各属性に対し、専門性の高い関連職業のリストを作成させる。
```
{
  "biology": {
    "microbiology": ["Clinical Microbiologist", "Pharmaceutical Microbiology Researcher", "Virologist", "Bacteriologist", "Mycologist", "Microbial Ecologist"],
    "genetics": [
  ...,
    "art and architecture": ["art history", "architectural theory", "visual culture", "digital art", "conservation science", "museum studies", "critical heritage studies"]
  }
}
```
  
↓  
category×jobで`{"subject": "biology", "group": "microbiology", "category": "virology", "job": "Clinical Microbiologist"}`というような複数のpersonaを作る。  
⇒ 2000弱しかできていない。



# 検証

自製したpersonaをseedにして問題を生成してみる


## 5. 適用カテゴリと数学スキルについて自分で深堀りさせる
  
 `topic`と`math_skill`について、プレースホルダーを使って深堀りし融合させ、問題の幅を広げる試みをする。

↓  
難易度：博士課程レベル


In [13]:
# persona_deiscription = "A scientist who specializes in virology and the study of ancient viruses. This persona is interested in the potential dangers of thawing permafrost and the revival of ancient viruses, and is likely to be part of a research team studying the potential impact of these viruses on human and animal health. This persona may also be familiar with the work of Jean-Michel Claverie, the lead author of the study."
# persona_deiscription = "A scientist who specializes in virology and the study of ancient viruses."


# p_seed = {"subject": "computer science", "group": "artificial intelligence", "category": "explainable AI", "job": "AIAI Robotics Researcher"}
p_seed = {"subject": "engineering", "group": "biomedical engineering", "category": "medical imaging", "job": "Biomechanics Engineer"}

persona_deiscription = f"A {p_seed["job"]} who specializes in {p_seed["group"]} and the study of {p_seed['category']}."

user_prompt = (
    "Create a math problem related to the following persona:\n\n"
    "{persona}\n\n"
    "Note:\n\n"
    "1. The math problem should be challenging and involve advanced mathematical skills and knowledge. Only top talents can solve it correctly.\n"
    "2. You should make full use of the persona description to create the math problem to ensure that the math problem is unique and specific to the persona.\n"
    '3. Your response should always start with "Math problem:". Your response should not include a solution to the created math problem.\n'
    # "4. Consider the advanced mathematical skills that are highly relevant to the persona description, and create specialised problems at master's or doctoral level that reflect these skills."
    # "4. Consider the advanced mathematical skills that are highly relevant to the persona description, and create specialised problems at leading researchers or doctoral-level level that reflect these skills.\n\n"
    "4. First, consider <topics> that require advanced mathematical skills closely related to persona descriptions and list the relevant <mathematical skills>."
    " Then, using your knowledge and experience of persona descriptions, create specialised master's or doctoral-level problems that combining the <topics> and <mathematical skills>, which are highly compatible with each other.\n\n"
    
    "The following issues should also be avoided when creating the math problem:\n"
    "- Insufficient information: Ploblems may lack the essential details needed to solve them, leaving them incomplete or ambiguous. For example, a trigonometry question might omit the necessary angles or distances.\n"
    "- Unsolvable or computationally intractable problems: Some problems are either unsolvable or require excessive brute-force calculations, which are impractical for evaluating reasoning abilities.\n"
    "- Nonsensical problems: Models sometimes produce problems that are logically inconsistent, confusing or ambiguous, such as a probability issue with unclear parameters or an impossible geometry scenario."
    " Inconsistent, confusing or ambiguous problems, such as a probability issue with unclear parameters or an impossible geometry scenario.\n"
    # "- Deceitful Solutions: Occasionally, models fabricate solutions to nonsensical or unsolvable problems, presenting incorrect logic as plausible reasoning.\n"
)

print(user_prompt.format(persona=persona_deiscription))


Create a math problem related to the following persona:

A Biomechanics Engineer who specializes in biomedical engineering and the study of medical imaging.

Note:

1. The math problem should be challenging and involve advanced mathematical skills and knowledge. Only top talents can solve it correctly.
2. You should make full use of the persona description to create the math problem to ensure that the math problem is unique and specific to the persona.
3. Your response should always start with "Math problem:". Your response should not include a solution to the created math problem.
4. First, consider <topics> that require advanced mathematical skills closely related to persona descriptions and list the relevant <mathematical skills>. Then, using your knowledge and experience of persona descriptions, create specialised master's or doctoral-level problems that combining the <topics> and <mathematical skills>, which are highly compatible with each other.

The following issues should also 

In [14]:
''' 問題の生成 '''

msgs:List[Dict[str, str]] = [
    {"role": "user", "content": user_prompt.format(persona=persona_deiscription)},
]

client = openai.OpenAI(api_key=api_key, base_url=base_url)

response = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=msgs,
)
generated_problem = response.choices[0].message.content
Math(generated_problem)

<IPython.core.display.Math object>

In [30]:
user_prompt = (
    "以下の問題を解いてください。解けない場合はその理由を説明してください。また、問題に不備がある場合は、修正可能であれば<修正済の問題文>を作成し、<修正済みの問題文>と修正箇所の説明を出力した上で、その問題を解いてください。\n"
    "{problem}\n\n"
    "問題が解けた場合は、この問題の数学的要求スキルとそれ以外の専門的な知識レベル（学部、修士、博士、学際レベルなど）などを要約し、問題の品質につて分析してください。\n"
    "<修正済みの問題文>以外はすべて日本語で回答してください\n"
)

# print(user_prompt.format(problem=generated_problem))

In [31]:
''' 生成した問題の評価 '''
msgs:List[Dict[str, str]] = [
    {"role": "user", "content": user_prompt.format(problem=generated_problem)},
]

client = openai.OpenAI(api_key=api_key, base_url=base_url)

response = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=msgs,
)
assessment_result = response.choices[0].message.content
Markdown(assessment_result)

### 問題の分析と解答

#### **数学的要求スキルと専門知識レベル**
- **数学的スキル**:  
  - ベクトル解析とテンソル解析（非ニュートン流体の粘度モデリング）。  
  - 非定常ナビエストークス方程式の導出と境界条件の設定（偏微分方程式）。  
  - フーリエ変換と逆問題（Volterra積分方程式）の解析。  
- **専門知識レベル**:  
  - **博士課程以上**：非ニュートン流体力学の高度なモデリング、生体医用画像処理の逆問題解析、微分方程式と積分方程式の厳密解導出に必要な数学的成熟度が必要。学際的（医用生体工学・応用数学・物理学）な背景が求められる。

---

#### **問題の品質評価**
1. **論理的一貫性**:  
   - 修正前は「定常流」と「脈動圧力勾配」の矛盾が不備だったが、修正後に解消。  
   - 非ニュートン流体の設定とMRI信号再構築の物理的妥当性は高い。  
2. **難易度**:  
   - 博士レベル以上の深い知識を要する。非ニュートン流体の微分方程式と逆問題を組み合わせた高度な内容。  
3. **独創性**:  
   - 生体血流と医用画像の実践的問題を数学的厳密性で扱っており、学術的・臨床的意義が高い。  

---

### <修正済の問題文>
**Biomechanical Modeling of Arterial Blood Flow with Non-Newtonian Viscosity and MRI Signal Reconstruction**  

A biomechanics engineer is studying blood flow dynamics in a stenosed (narrowed) artery using medical imaging. Blood is modeled as a non-Newtonian fluid with shear-thinning behavior, governed by the Carreau-Yasuda constitutive equation:  
\[
\mu(\dot{\gamma}) = \mu_\infty + (\mu_0 - \mu_\infty)\left[1 + (\lambda \dot{\gamma})^a\right]^{(n-1)/a},
\]  
where \(\dot{\gamma} = \sqrt{2 \mathbf{D} : \mathbf{D}}\) is the shear rate, \(\mathbf{D}\) is the strain-rate tensor, \(\mu_0 = 0.12 \, \text{Pa·s}\), \(\mu_\infty = 0.0035 \, \text{Pa·s}\), \(\lambda = 8.2 \, \text{s}\), \(a = 2\), and \(n = 0.356\).  

The axisymmetric arterial stenosis is approximated by the lumen radius function:  
\[
R(z) = R_0 \left[1 - \beta \exp\left(-\frac{(z - z_0)^2}{2\sigma^2}\right)\right],
\]  
where \(R_0 = 3.0 \, \text{mm}\), \(\beta = 0.6\), \(\sigma = 2.5 \, \text{mm}\), and \(z_0 = 10.0 \, \text{mm}\).  

1. **Fluid Dynamics Modeling**:  
   Using the continuity and momentum equations for incompressible flow in cylindrical coordinates \((r, \theta, z)\), derive the simplified Navier-Stokes equations for laminar flow under pulsatile pressure gradient \(\frac{\partial p}{\partial z}(t) = -P_0 \left[1 + 0.5 \sin(\omega t)\right]\), where \(P_0 = 500 \, \text{Pa/m}\) and \(\omega = 2\pi \, \text{rad/s}\). Assume axial symmetry (\(\partial/\partial \theta = 0\)) and no swirl. Formulate the boundary conditions for velocity at the arterial wall and symmetry axis.  

2. **MRI Velocity Encoding**:  
   Phase-contrast MRI measures velocity fields by encoding tissue displacement into signal phase shifts. The phase difference \(\Delta \phi\) is proportional to the velocity \(u_z(r,z,t)\):  
   \[
   \Delta \phi(r,z) = \gamma G \int_0^{\Delta t} u_z(r,z,t) \, dt,
   \]  
   where \(\gamma = 267.5 \times 10^6 \, \text{rad·s}^{-1}\text{T}^{-1}\) is the gyromagnetic ratio, \(G = 40 \, \text{mT/m}\) is the gradient strength, and \(\Delta t = 20 \, \text{ms}\).  
   Using the velocity profile derived in part 1, compute the Fourier transform of \(\Delta \phi(r,z)\) with respect to \(z\) at \(r = 0\) (centerline). Show that the inverse problem of reconstructing \(u_z(0,z,t)\) from \(\Delta \phi(0,z)\) requires solving a Volterra integral equation of the first kind.  

**修正箇所の説明**:  
- 問題1の「steady, laminar flow」から「steady」を削除。脈動圧力勾配（時間依存項）と定常流の矛盾を解消するため。非定常流を暗黙に仮定。  

---

### **解答の試み**
問題の複雑さから、完全な解析解は現実的でないため、以下は方針を示す。

#### **1. 非定常ナビエストークス方程式の導出**
- **連続の式**:  
  \[
  \frac{1}{r}\frac{\partial (r u_r)}{\partial r} + \frac{\partial u_z}{\partial z} = 0.
  \]  
- **運動量方程式（z成分）**:  
  \[
  \rho \left( \frac{\partial u_z}{\partial t} + u_r \frac{\partial u_z}{\partial r} + u_z \frac{\partial u_z}{\partial z} \right) = -\frac{\partial p}{\partial z} + \frac{1}{r}\frac{\partial}{\partial r} \left( r \mu(\dot{\gamma}) \frac{\partial u_z}{\partial r} \right).
  \]  
  （仮定：radial速度 \(u_r\) は微小、粘性項の支配項のみ保持。）  
- **境界条件**:  
  - 動脈壁 \(r = R(z)\): \(u_z = 0, \, u_r = 0\).  
  - 中心軸 \(r = 0\): \(\frac{\partial u_z}{\partial r} = 0, \, u_r = 0\).  

#### **2. MRI信号の逆問題**
- **位相差のフーリエ変換**:  
  \[
  \mathcal{F}[\Delta \phi(0,z)] = \gamma G \int_{-\infty}^\infty \left( \int_0^{\Delta t} u_z(0,z,t) dt \right) e^{-ikz} dz.
  \]  
  積分順序を交換し、  
  \[
  \mathcal{F}[\Delta \phi] = \gamma G \int_0^{\Delta t} \mathcal{F}[u_z(0,z,t)] dt.
  \]  
  逆問題では、観測データ \(\mathcal{F}[\Delta \phi]\) から \(u_z\) を再構築するため、積分方程式（Volterra型）を解く必要がある。

In [ ]:
user_prompt = (
    "以下の問題を解いてください。解けない場合はその理由を説明してください:<要約>。また、問題に不備がある場合は、修正可能であれば<修正済の問題文>を作成し、<修正済みの問題文>と修正箇所の説明:<修正点>を出力した上で、その問題を解いてください。\n"
    "{problem}\n\n"
    "問題が解けた場合はその最終回答のみを<回答>に格納してください、回答が複数ある場合は[回答1,回答2,...]とリスト形式の<回答>としてください。\n"
    "また問題が解けが場合は、数学的要求スキルとそれ以外の専門的な知識レベル（学部、修士、博士、学際レベルなど）などを要約し、問題の品質につて分析してください\n"
    "<修正済みの問題文>以外はすべて日本語で回答してください\n"
    "以下のフォーマットで出力し、他のテキストは含めないでください。\n"
    "json format:\n\n{{'revised_problem':<修正済みの問題文>, 'revise_point':<修正点>, 'answer':<回答>, 'summary':<要約>}}"
)

# print(user_prompt.format(problem=generated_problem))

In [26]:
user_prompt

"以下の問題を解いてください。解けない場合はその理由を説明してください:<要約>。また、問題に不備がある場合は、修正可能であれば<修正済の問題文>を作成し、<修正済みの問題文>と修正箇所の説明:<修正点>を出力した上で、その問題を解いてください。\n{problem}\n\n問題が解けた場合はその最終回答のみを<回答>に格納してください、回答が複数ある場合は[回答1,回答2,...]とリスト形式の<回答>としてください。\nまた問題が解けが場合は、数学的要求スキルとそれ以外の専門的な知識レベル（学部、修士、博士、学際レベルなど）などを要約し、問題の品質につて分析してください\n<修正済みの問題文>以外はすべて日本語で回答してください\njson format:\n\n{{'revised_problem':<修正済みの問題文>, 'revise_point':<修正点>, 'answer':<回答>, 'summary':<要約>}}"

In [29]:

''' 生成した問題の評価 '''
msgs:List[Dict[str, str]] = [
    {"role": "user", "content": user_prompt.format(problem=generated_problem)},
]

client = openai.OpenAI(api_key=api_key, base_url=base_url)

response = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=msgs,
)
assessment_result = response.choices[0].message.content
Markdown(assessment_result)

```json
{
  "revised_problem": "**Biomechanical Modeling of Arterial Blood Flow with Non-Newtonian Viscosity and MRI Signal Reconstruction**\n\nA biomechanics engineer is studying blood flow dynamics in a stenosed (narrowed) artery using medical imaging. Blood is modeled as a non-Newtonian fluid with shear-thinning behavior, governed by the Carreau-Yasuda constitutive equation:\n\\[\n\\mu(\\dot{\\gamma}) = \\mu_\\infty + (\\mu_0 - \\mu_\\infty)\\left[1 + (\\lambda \\dot{\\gamma})^a\\right]^{(n-1)/a},\n\\]\nwhere \\(\\dot{\\gamma} = \\sqrt{2 \\mathbf{D} : \\mathbf{D}}\\) is the shear rate, \\(\\mathbf{D}\\) is the strain-rate tensor, \\(\\mu_0 = 0.12 \\, \\text{Pa·s}\\), \\(\\mu_\\infty = 0.0035 \\, \\text{Pa·s}\\), \\(\\lambda = 8.2 \\, \\text{s}\\), \\(a = 2\\), and \\(n = 0.356\\).\n\nThe axisymmetric arterial stenosis is approximated by the lumen radius function:\n\\[\nR(z) = R_0 \\left[1 - \\beta \\exp\\left(-\\frac{(z - z_0)^2}{2\\sigma^2}\\right)\\right],\n\\]\nwhere \\(R_0 = 3.0 \\, \\text{mm}\\), \\(\\beta = 0.6\\), \\(\\sigma = 2.5 \\, \\text{mm}\\), and \\(z_0 = 10.0 \\, \\text{mm}\\).\n\n1. **Fluid Dynamics Modeling**:\n   Using the continuity and momentum equations for incompressible flow in cylindrical coordinates \\((r, \\theta, z)\\), derive the simplified Navier-Stokes equations for **pulsatile**, laminar flow under pulsatile pressure gradient \\(\\frac{\\partial p}{\\partial z}(t) = -P_0 \\left[1 + 0.5 \\sin(\\omega t)\\right]\\), where \\(P_0 = 500 \\, \\text{Pa/m}\\) and \\(\\omega = 2\\pi \\, \\text{rad/s}\\). Assume axial symmetry (\\(\\partial/\\partial \\theta = 0\\)) and no swirl. Formulate the boundary conditions for velocity at the arterial wall and symmetry axis.\n\n2. **MRI Velocity Encoding**:\n   Phase-contrast MRI measures velocity fields by encoding tissue displacement into signal phase shifts. The phase difference \\(\\Delta \\phi\\) is proportional to the velocity \\(u_z(r,z,t)\\):\n   \\[\n   \\Delta \\phi(r,z) = \\gamma G \\int_0^{\\Delta t} u_z(r,z,t) \\, dt,\n   \\]\n   where \\(\\gamma = 267.5 \\times 10^6 \\, \\text{rad·s}^{-1}\\text{T}^{-1}\\) is the gyromagnetic ratio, \\(G = 40 \\, \\text{mT/m}\\) is the gradient strength, and \\(\\Delta t = 20 \\, \\text{ms}\\).\n   Using the velocity profile derived in part 1, compute the Fourier transform of \\(\\Delta \\phi(r,z)\\) with respect to \\(z\\) at \\(r = 0\\) (centerline). Show that the inverse problem of reconstructing \\(u_z(0,z,t)\\) from \\(\\Delta \\phi(0,z)\\) requires solving a Volterra integral equation of the first kind.\n\n**Mathematical Skills Required**:\n- Advanced tensor calculus for non-Newtonian fluid constitutive modeling.\n- Partial differential equations (Navier-Stokes simplification for pulsatile flow).\n- Fourier transforms and integral equations for inverse problems in signal reconstruction.\n- Analysis of Volterra integral equations.",
  "revise_point": "問題1の **steady** を **pulsatile** に修正: 元の定常流れ（steady flow）の仮定が、与えられた時間依存圧力勾配（sin波を含む）と矛盾していたため、非定常流れ（pulsatile flow）に修正しました。これによりNavier-Stokes方程式の導出が物理的に妥当なものになります。",
  "answer": [],
  "summary": "この問題は以下の理由で高度すぎて解けません:\n- **学際性**: 非ニュートン流体力学（Carreau-Yasudaモデル）、脈動流のNavier-Stokes方程式の導出、MRI信号処理（フーリエ変換とVolterra方程式）が組み合わさり、博士レベルの専門知識が必要です。\n- **数学的難度**: 非線形偏微分方程式の導出と解析解の構築は困難です。\n- **逆問題の厳密性**: Volterra積分方程式の解析は数値解法必須で、信号のサンプリング条件が不明な点も障壁です。\n修正後も、非定常流れにおける非ニュートン流体の運動方程式を導出するには、流体力学とテンソル解析の深い理解が必要です。MRI信号の再構築は数学的な逆問題の高度なスキルを要求します。"
}
```